# 🔧 Xenon v1.0.0: XML Repair for LLM-Generated XML

Welcome to the interactive Xenon demo! This notebook shows you how to repair malformed XML commonly generated by Large Language Models.

**What is Xenon?**
- **Secure-by-default**: Explicit trust levels for input sources.
- **Zero Dependencies**: Uses only Python Standard Library.
- **LLM-Focused**: Specifically designed to handle common LLM XML generation failures.
- **Accurate Reporting**: Get detailed, accurate reports on every repair made.

**Open in Colab**: [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MarsZDF/xenon/blob/main/xenon_demo.ipynb)

## 📦 Installation

Install Xenon from PyPI:

In [ ]:
# Install Xenon from PyPI
!pip install -q elemental-xenon

from xenon import (
    repair_xml_safe,
    repair_xml_with_report,
    parse_xml_safe,
    format_xml,
    TrustLevel, # New in v1.0.0
    RepairType
)

print("✅ Xenon installed successfully!")

---

# 🎯 The 4 Major LLM XML Failure Modes

Xenon handles the most common ways LLMs break XML. Notice the mandatory `trust` parameter in all examples.

## 1️⃣ Truncation / Cut-off

LLMs run out of tokens mid-tag and leave XML incomplete:

In [ ]:
# LLM output that got cut off
truncated = '<root><user name="alice"><address><city>San Francisco'

print("❌ BROKEN XML:")
print(truncated)
print()

# Xenon repairs it
repaired = repair_xml_safe(truncated, trust=TrustLevel.UNTRUSTED)
print("✅ REPAIRED XML:")
print(repaired)

## 2️⃣ Conversational Fluff

LLMs wrap XML in conversational text:

In [ ]:
# LLM adds unnecessary commentary
fluff = '''
Sure! Here's the XML you requested:

<root>
    <message>Hello World</message>
    <status>success</status>
</root>

Hope this helps! Let me know if you need anything else.
'''

print("❌ MESSY OUTPUT:")
print(fluff)
print()

# Xenon extracts just the XML
repaired = repair_xml_safe(fluff, trust=TrustLevel.UNTRUSTED)
print("✅ CLEAN XML:")
print(repaired)

## 3️⃣ Malformed Attributes

LLMs forget to quote attribute values:

In [ ]:
# Missing quotes around attributes
unquoted = '<product id=12345 category=electronics price=299.99>Laptop</product>'

print("❌ INVALID ATTRIBUTES:")
print(unquoted)
print()

# Xenon adds quotes
repaired = repair_xml_safe(unquoted, trust=TrustLevel.UNTRUSTED)
print("✅ VALID ATTRIBUTES:")
print(repaired)

## 4️⃣ Unescaped Entities

LLMs forget to escape special characters like `&` and `<`:

In [ ]:
# Special characters not escaped
unescaped = '<description>Price: $5 < $10 & shipping included</description>'

print("❌ UNESCAPED ENTITIES:")
print(unescaped)
print()

# Xenon escapes them
repaired = repair_xml_safe(unescaped, trust=TrustLevel.UNTRUSTED)
print("✅ PROPERLY ESCAPED:")
print(repaired)

---

# ✨ NEW in v1.0.0: Accurate Repair Analysis

See exactly what Xenon fixed with detailed, accurate repair reports. No more guesswork!

In [ ]:
# Malformed XML with multiple issues
messy = '''
Here's your data:

<users>
    <user id=1001 role=admin>
        <name>John Smith & Associates</name>
        <email>john@example.com</email>
        <status>Active & Verified
'''

# Get detailed report of what was fixed
repaired, report = repair_xml_with_report(messy, trust=TrustLevel.UNTRUSTED)

print(f"📊 Found {len(report.actions)} repairs to make:")
for action in report.actions:
    print(f"  - Type: {action.repair_type.value}")
    print(f"    Description: {action.description}")
    if action.before:
        print(f"    Before: {action.before}")
        print(f"    After:  {action.after}")
print("\n" + "-"*20 + "\n")
print("✅ REPAIRED XML:")
print(repaired)

## View Changes as Diff

See before/after comparison:

In [ ]:
# Show unified diff
print("📝 UNIFIED DIFF:")
print(report.to_unified_diff())

---

# 🎨 XML Formatting

Format XML for readability or storage:

In [ ]:
# Compact XML
compact = '<root><item>test</item><another>data</another></root>'

print("📝 ORIGINAL (compact):")
print(compact)
print()

# Pretty-print for readability
pretty = format_xml(compact, style='pretty')
print("✨ PRETTY-PRINTED:")
print(pretty)

## Repair + Format in One Step

In [ ]:
# Broken XML that needs formatting
broken = '<root><item>test</item><another>data'

# Repair AND format in one call
result = repair_xml_safe(broken, trust=TrustLevel.TRUSTED, format_output='pretty')

print("📥 INPUT:")
print(broken)
print()
print("📤 OUTPUT (repaired + formatted):")
print(result)

---

# 📊 Parse to Dictionary

Convert repaired XML to Python dictionaries:

In [ ]:
import json

# Malformed XML from LLM
malformed = '<response status=success><data count=3><item>Apple</item><item>Banana</item><item>Orange'

print("📝 Input:")
print(malformed)
print()

# Parse directly (repairs automatically)
data = parse_xml_safe(malformed, trust=TrustLevel.UNTRUSTED)

print("📦 Parsed Dictionary:")
print(json.dumps(data, indent=2))
print()

# Access data easily
print("🎯 Extracted Values:")
print(f"  Status: {data['response']['@attributes']['status']}")
print(f"  Count: {data['response']['data']['@attributes']['count']}")
print(f"  Items: {data['response']['data']['item']}")

---

# 🛡️ Security Audit Logging

For SOC 2 compliance and security monitoring, log all repair actions and threats:

In [ ]:
from xenon.audit import AuditLogger

# 1. Configure Logger
logger = AuditLogger()

# 2. Process potentially malicious input
risky_input = '<?php system("rm -rf /"); ?> <script>alert(1)</script> <user>Alice</user>'

print("📥 RISKY INPUT:")
print(risky_input)
print()

# 3. Repair with logging enabled
safe_output = repair_xml_safe(
    risky_input,
    trust=TrustLevel.UNTRUSTED,
    audit_logger=logger
)

print("📤 SANITIZED OUTPUT:")
print(safe_output)
print()

# 4. View Audit Log (JSON format)
print("📋 AUDIT LOG ENTRY:")
import json
print(json.dumps(logger.to_json(), indent=2))

---

# 🦜🔗 LangChain Integration

Use Xenon as a drop-in OutputParser for LangChain:

In [ ]:
try:
    from xenon.integrations.langchain import XenonXMLOutputParser
    
    # Initialize parser
    parser = XenonXMLOutputParser(
        trust=TrustLevel.UNTRUSTED,
        return_dict=True  # Set False to get XML string
    )
    
    # Simulate LLM output
    llm_output = "Here is the data: <user id=123>Bob</user>"
    
    # Parse
    result = parser.parse(llm_output)
    
    print("✅ LangChain Parser Result:")
    print(json.dumps(result, indent=2))

except ImportError:
    print("⚠️ LangChain not installed in this environment")

---

# 🎮 Interactive Playground

Try your own malformed XML below!

In [ ]:
# ✏️ Edit this XML and run the cell to see how Xenon repairs it!

your_xml = '''
Sure, here's the XML:

<config>
    <database host=localhost port=5432>
        <credentials user=admin password=secret123
'''

print("📥 YOUR INPUT:")
print(your_xml)
print()
print("=" * 60)
print()

# Repair with detailed report
repaired, report = repair_xml_with_report(your_xml, trust=TrustLevel.UNTRUSTED, format_output='pretty')

print("🔧 REPAIRS MADE:")
for action in report.actions:
    print(f"  • {action.description} (Type: {action.repair_type.value})")
print()
print("=" * 60)
print()

print("📤 XENON OUTPUT:")
print(repaired)